This notebook compares the gnubg bearoff database to our calculations. It assumes that you have generated the data files with these commands

`./compute.py 15 6`

`./gnubg_to_store.py ~/gnubg/gnubg-1.06.002/`

For the latter, it shoudl be a directory where you have the source of gnubg downloaded and made


In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import numpy as np

import board
import strategy

In [4]:
!./compute.py 6 3
!ls -l data/
import h5py
with h5py.File('data/bgend_store_6_3.hdf5', "r") as f:
    pass
!ls -l data/

Starting compute on 84 boards
total 23616
-rw-rw-r-- 1 pfr pfr      800 Mar 23 21:01 bgend_store_15_6.hdf5
-rw-rw-r-- 1 pfr pfr    37712 Mar 23 21:13 bgend_store_6_3.hdf5
-rw-rw-r-- 1 pfr pfr 24134504 Mar 22 21:47 gnubg_store_15_6.hdf5
total 23616
-rw-rw-r-- 1 pfr pfr      800 Mar 23 21:01 bgend_store_15_6.hdf5
-rw-rw-r-- 1 pfr pfr    37712 Mar 23 21:13 bgend_store_6_3.hdf5
-rw-rw-r-- 1 pfr pfr 24134504 Mar 22 21:47 gnubg_store_15_6.hdf5


In [10]:
%%time
our_store = strategy.DistributionStore.load_hdf5('data/bgend_store_6_3.hdf5')
gnubg_store = strategy.DistributionStore.load_hdf5('data/gnubg_store_15_6.hdf5')

CPU times: user 11.1 s, sys: 288 ms, total: 11.4 s
Wall time: 11.4 s


In [11]:
print(len(our_store.distribution_map))
print(len(gnubg_store.distribution_map))

84
54264


In [18]:
for board_id in our_store.distribution_map:
    ours = our_store.distribution_map[board_id]
    #theirs = gnubg_store.distribution_map[board_id]
    theirs = our_store.distribution_map[board_id]
    if len(ours) != len(theirs):
        print('Board ID {}, different lengths, ours={}, theirs={}'.format(board_id, ours, theirs))
        continue
    # The atol of 1e-5 is because of the limited precision from going through the text parsing from gnubg
    if nnp.allclose(ours.dist, theirs.dist, atol=1e-5):
        print('Board ID {}, different lengths, ours={}, theirs={}'.format(board_id, ours, theirs))
        continue